In [ ]:
import os

import numpy
import pandas as pd
import pickle
import seaborn as sns
from textwrap import wrap
from matplotlib import pyplot as plt
from matplotlib import ticker
from scipy.stats import mode

from hera_cal.redcal import get_reds

from red_likelihood import fltBad, groupBls, group_data, gVis, relabelAnts
from red_utils import find_flag_file, find_nearest, find_rel_df, find_zen_file, \
get_bad_ants, match_lst, split_rel_results

In [ ]:
plt.rcParams['figure.figsize'] = (12, 8)
%matplotlib inline

clip_pctile = 97 # for heatmaps to set vmax

In [ ]:
JD = 2458098.43869
pol = 'ee'

In [ ]:
zen_fn = find_zen_file(JD)
bad_ants = get_bad_ants(zen_fn)
flags_fn = find_flag_file(JD, 'first')

In [ ]:
hdraw, RedG, cMData = group_data(zen_fn, pol, None, None, bad_ants, flags_fn)
cData_1 = cMData.filled()

# Difference in adjacent visibilities as an estimate of noise

###  Selecting visibilities for a given baseline type and frequency

In [ ]:
freq_channel = 620

In [ ]:
reds = fltBad(get_reds(hdraw.antpos, pols=[pol]), bad_ants)
bl_types = RedG[:, 0]
slct_bl_type_id = mode(bl_types)[0][0] # selecting modal value for baseline type
slct_bl_type = reds[slct_bl_type_id][0]
slct_idxs = numpy.where(bl_types == slct_bl_type_id)[0]
print('Selecting visibilities with baseline type {} that are redundant with '\
      'baseline {}, of which there are {}'.format(slct_bl_type_id, slct_bl_type, \
      slct_idxs.size))

In [ ]:
slct_vis_t = numpy.squeeze(cData_1[numpy.ix_([freq_channel], \
                                             numpy.arange(cData_1.shape[1]), slct_idxs)])

In [ ]:
vis_amp_t = numpy.abs(slct_vis_t)

fig, ax = plt.subplots(figsize=(11, 7))
plt.plot(vis_amp_t, alpha=0.5)
plt.plot(numpy.median(vis_amp_t, axis=1), linewidth=3)
plt.xlabel('Time integration')
plt.ylabel('Visibility amplitude')
fig.tight_layout()
plt.show()

In [ ]:
df_t = pd.DataFrame(vis_amp_t).stack().reset_index()
df_t.rename(columns={'level_0': 'time_int', 'level_1': 'bl', 0: 'vis_amp'}, inplace=True)

fig, ax = plt.subplots(figsize=(11, 7))
ax = sns.lineplot(x='time_int', y='vis_amp', data=df_t, ci='sd', linewidth=3)
plt.xlabel('Time integration')
plt.ylabel('Visibility amplitude')
fig.tight_layout()
plt.show()

In [ ]:
vis_phase_t = numpy.angle(slct_vis_t)

fig, ax = plt.subplots(figsize=(11, 7))
plt.plot(vis_phase_t)
plt.xlabel('Time integration')
plt.ylabel('Visibility phase')
fig.tight_layout()
plt.show()

## Adjacent time integrations

For a given baseline, one can compare the visibilities between one time integration and the next to get an estimate of the system noise, since the difference between adjacent visibilities will encapsulate both the noise and the difference in the observed sky (due to a slight drift). The statistics of the difference in visibilities therefore provides an upper bound on the noise of the visibilities.

We take the standard deviation of the different between visibilities adjacent in time for an entire dataset, as a proxy for the noise. This must first be done per baseline and per frequency. The noise for redundant baselines can then be combined through propagation of error considerations.

n.b. the variance of a complex random variable $z$ is equal to the sum of the variances of its real and imaginary parts:

$$ \mathrm{Var}[z] = \mathrm{Var}[\Re(z)] + \mathrm{Var}[\Im(z)] $$

which we use when calculating the standard deviation of complex visibilities.

In [ ]:
vis_diffs = numpy.empty((cData_1.shape[0], cData_1.shape[1]-1, \
                         cData_1.shape[2]), dtype=complex)
noise_std = numpy.empty((cData_1.shape[0], cData_1.shape[2]))
mean_vis_amp = numpy.empty_like(noise_std)
for bl in range(cData_1.shape[2]):
    for freq in range(cData_1.shape[0]):
        vdiff = numpy.asarray([t - s for s, t in zip(cData_1[freq, :, bl], \
                                                     cData_1[freq, 1:, bl])])
        vis_diffs[freq, :, bl] = vdiff
        noise_std[freq, bl] = numpy.sqrt(numpy.var(vdiff.real) + \
                                             numpy.var(vdiff.imag))
        mean_vis_amp[freq, bl] = numpy.mean(numpy.abs(cData_1[freq, :, bl]))

We add errors in quadrature when considering the noise across a redundant baseline type, assuming an error covariance of zero (independent measurements and uncorrelated errors).

$$\sigma_{\text{red_group}} = \frac{1}{N} \sqrt{\sum_i{\sigma_i^2}} $$

In [ ]:
no_unq_bls = numpy.unique(bl_types).size
red_noise = numpy.empty((cData_1.shape[0], no_unq_bls))
red_vis_amp = numpy.empty_like(red_noise)

for bl_type in range(no_unq_bls):
    group_idxs = numpy.where(bl_types == bl_type)[0]
    grouped_noise = noise_std[:, group_idxs]
    grouped_vis_amp = mean_vis_amp[:, group_idxs]
    red_noise[:, bl_type] = numpy.sqrt(numpy.sum(numpy.square(grouped_noise), axis=1)) \
                            / grouped_noise.shape[1]
    red_vis_amp[:, bl_type] = numpy.mean(grouped_vis_amp, axis=1)

### Single baseline: {{reds[slct_bl_type][0]}}

#### Single frequency: channel {{freq_channel}}

In [ ]:
bl_grp_id = 0 # ID of baseline within its group
bl_id = slct_idxs[bl_grp_id]

vis_bl_t = cData_1[:, :, bl_id]
print('From the {} baselines of type {} selected, only consider baseline {}'\
      .format(slct_idxs.size, slct_bl_type, \
              reds[slct_bl_type_id][bl_grp_id]))

vis_diffs_t = vis_diffs[:, :, bl_id]
noise_std_t = noise_std[freq_channel, bl_id]
mean_vis_amp_t = mean_vis_amp[freq_channel, bl_id]
print('Upper bound on noise, by comparing adjacent visibilities in time for '\
      'frequency channel {} is {}, which is {}% of the mean visibility amplitude'.\
      format(freq_channel, round(noise_std_t, 5), round(100*noise_std_t/mean_vis_amp_t, 1)))

#### Noise per frequency

In [ ]:
vis_bl_tf = cData_1[..., bl_id]
vis_diffs_tf = vis_diffs[..., bl_id]
noise_std_tf = noise_std[..., bl_id]
mean_vis_amp_tf = mean_vis_amp[..., bl_id]

In [ ]:
fig, ax = plt.subplots(figsize=(14,7))
diffs = numpy.abs(vis_diffs_tf).transpose()
ax = sns.heatmap(diffs, vmax=numpy.ceil(numpy.nanpercentile(diffs, clip_pctile)*100)/100)
ax.xaxis.set_major_locator(ticker.MultipleLocator(50))
ax.xaxis.set_major_formatter(ticker.ScalarFormatter())
ax.yaxis.set_major_locator(ticker.MultipleLocator(5))
ax.yaxis.set_major_formatter(ticker.ScalarFormatter())
plt.xlabel('Frequency channel')
plt.ylabel('Time integration')
plt.title('Residual between adjacent visibilities (in time)')
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(11, 7))
plt.plot(noise_std_tf)
ax.set_yscale('log')
plt.xlabel('Frequency channel')
plt.ylabel('Log-noise')
plt.title('Log of the standard deviation of residuals between adjactent visibilities '\
          '(in time)')
fig.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(11, 7))
plt.plot(noise_std_tf/mean_vis_amp_tf)
plt.xlabel('Frequency channel')
plt.ylabel('Fractional noise')
plt.title('Standard deviation of residuals between adjactent visibilities '\
          '(in time) divided by the mean visibility amplitude')
fig.tight_layout()
plt.show()

### Redundant baseline group

In [ ]:
print('Considering baselines of type {}'.format(slct_bl_type))

In [ ]:
vis_diffs_tfb = cData_1[..., slct_idxs]
noise_std_tfb = noise_std[..., slct_idxs]
mean_vis_amp_tfb = mean_vis_amp[..., slct_idxs]

In [ ]:
fig, ax = plt.subplots(figsize=(11, 7))
for bl in range(noise_std_tfb.shape[1]):
    plt.plot(noise_std_tfb[:, bl])
ax.set_yscale('log')
plt.xlabel('Frequency channel')
plt.ylabel('Log-noise')
plt.title('Log of the standard deviation of residuals between adjactent visibilities '\
          '(in time) for different baselines of type')
fig.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(11, 7))
for bl in range(noise_std_tfb.shape[1]):
    plt.plot(noise_std_tfb[:, bl]/mean_vis_amp_tfb[:, bl])
plt.xlabel('Frequency channel')
plt.ylabel('Fractional noise')
plt.title('\n'.join(wrap('Standard deviation of residuals between adjactent visibilities '\
          '(in time) divided by the mean visibility amplitude, for baselines redundant '\
          'with {}'.format(slct_bl_type), 110)))
plt.ylim((0, 5))
fig.tight_layout()
plt.show()

In [ ]:
frac_noise_blgroup = noise_std_tfb/mean_vis_amp_tfb
fig, ax = plt.subplots(figsize=(14,7))
ax = sns.heatmap((frac_noise_blgroup).transpose(), \
                 vmax=numpy.ceil(numpy.nanpercentile(frac_noise_blgroup, clip_pctile)*100)/100)
ax.xaxis.set_major_locator(ticker.MultipleLocator(50))
ax.xaxis.set_major_formatter(ticker.ScalarFormatter())
ax.yaxis.set_major_locator(ticker.MultipleLocator(5))
ax.yaxis.set_major_formatter(ticker.ScalarFormatter())
plt.xlabel('Frequency channel')
plt.ylabel('Baseline')
plt.title('\n'.join(wrap('Standard deviation of residuals between adjactent visibilities '\
          '(in time) divided by the mean visibility amplitude, for baselines in redundant group {}'\
          .format(slct_bl_type), 100)))
plt.show()

#### Noise across redundant group

In [ ]:
red_noise_g = red_noise[:, slct_bl_type_id]

In [ ]:
fig, ax = plt.subplots(figsize=(11, 7))
plt.plot(red_noise_g)
ax.set_yscale('log')
plt.xlabel('Frequency channel')
plt.ylabel('Log-noise')
plt.title('\n'.join(wrap('Log of the combined standard deviation of residuals between adjactent visibilities '\
          '(in time) for baseline group of type {}'.format(slct_bl_type), 110)))
fig.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(11, 7))
plt.plot(red_noise_g/numpy.mean(mean_vis_amp[:, slct_idxs], axis=1))
plt.xlabel('Frequency channel')
plt.ylabel('Fractional noise')
plt.title('\n'.join(wrap('Standard deviation of residuals between adjactent visibilities '\
          '(in time) divided by the mean visibility amplitude, for baselines redundant '\
          'with {}'.format(slct_bl_type), 110)))
plt.ylim(bottom=0, top=1)
fig.tight_layout()
plt.show()

### Noise across baselines

In [ ]:
frac_noise_all = (red_noise/red_vis_amp).transpose()

fig, ax = plt.subplots(figsize=(14,7))
ax = sns.heatmap(frac_noise_all, \
                 vmax=numpy.ceil(numpy.nanpercentile(frac_noise_all, clip_pctile)*100)/100)
ax.xaxis.set_major_locator(ticker.MultipleLocator(50))
ax.xaxis.set_major_formatter(ticker.ScalarFormatter())
ax.yaxis.set_major_locator(ticker.MultipleLocator(5))
ax.yaxis.set_major_formatter(ticker.ScalarFormatter())
plt.xlabel('Frequency channel')
plt.ylabel('Baseline type')
plt.title('\n'.join(wrap('Standard deviation of residuals between adjactent visibilities '\
          '(in time) divided by the mean visibility amplitude, for all baseline types '\
          , 100)))
plt.show()

The lower baseline type IDs correspond to shorter baselines, with baseline length increase (and number of baselines decreasing) with as the baseline type ID increases. We find that the shorter baselines higher frequency channels have the lowest noise out of our dataset.

Note that the frequency range in the closure phase analysis is $100-200$ MHz, for the shortest triads ($14$ m and $28$ m)

## Same LAST on different JDs

An alternative estimate of the noise is by looking at visibilities from different JDs that match in LAST, although the visibilities on different JDs may vastly differ due to the variability of the instrumental gains and other effects (e.g. ionosphere).

In [ ]:
JD_2a= match_lst(JD, 2458099) # finding the JD_time of the zen_file
# that matches the LAST of the first dataset used
zen_fn2a = find_zen_file(JD_2a)
bad_ants2a = get_bad_ants(zen_fn2a)
flags_fn2a = find_flag_file(JD_2a, 'first')

In [ ]:
last_df = pd.read_pickle('jd_lst_map_idr2.pkl')

next_row = numpy.where(last_df['JD_time'] == JD_2a)[0][0] + 1
JD_2b = last_df.iloc[next_row]['JD_time']
zen_fn2b = find_zen_file(JD_2b)
bad_ants2b = get_bad_ants(zen_fn2b)
flags_fn2b = find_flag_file(JD_2b, 'first')

In [ ]:
last_df = pd.read_pickle('jd_lst_map_idr2.pkl')

next_row = numpy.where(last_df['JD_time'] == JD_2a)[0][0] + 1
JD_2b = last_df.iloc[next_row]['JD_time']
zen_fn2b = find_zen_file(JD_2b)
bad_ants2b = get_bad_ants(zen_fn2b)
flags_fn2b = find_flag_file(JD_2b, 'first')

In [ ]:
last1 = last_df[last_df['JD_time'] == JD]['LASTs'].values[0]
last2 = last_df[last_df['JD_time'] == JD_2a]['LASTs'].values[0]
_, offset = find_nearest(last2, last1[0])

In [ ]:
_, _, cMData = group_data(zen_fn2a, pol, freq_channel, None, bad_ants2a, flags_fn2a)
cData_2a = numpy.squeeze(cMData.filled())[offset:]

_, _, cMData = group_data(zen_fn2b, pol, freq_channel, None, bad_ants2b, flags_fn2b)
cData_2b = numpy.squeeze(cMData.filled())[:offset]

In [ ]:
cData_2 = numpy.concatenate((cData_2a, cData_2b))
slct_vis_j = cData_2[:, slct_idxs]

In [ ]:
vis_amp_j = numpy.abs(slct_vis_j)

fig, ax = plt.subplots(figsize=(11, 7))
plt.plot(vis_amp_j, alpha=0.5)
plt.plot(numpy.median(vis_amp_j, axis=1), linewidth=3)
plt.xlabel('Time integration')
plt.ylabel('Visibility amplitude')
fig.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(11, 7))
plt.plot(numpy.median(vis_amp_t, axis=1))
plt.plot(numpy.median(vis_amp_j, axis=1))
plt.xlabel('Time integration')
plt.ylabel('Visibility amplitude')
plt.title('Median visibility amplitude on separate JD days but same LAST')
fig.tight_layout()
plt.show()

In [ ]:
vis_bl_j = slct_vis_j[:, 0]
vis_diff_j = numpy.asarray([t - s for s, t in zip(vis_bl_j, vis_bl_j[1:])])

In [ ]:
noise_std_j = numpy.sqrt(numpy.var(vis_diff_j.real) + numpy.var(vis_diff_j.imag))
mean_vis_amp_j = numpy.mean(numpy.abs(vis_bl_j))
print('Upper bound on noise, by comparing adjacent visibilities in time at the same '\
      'frequency is {}, which is {}% of the mean visibility amplitude'.\
      format(round(noise_std_j, 5), round(100*noise_std_j/mean_vis_amp_j, 1)))